In [ ]:
#Neural Network using Nested Design using Fashion minist dataset 

In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils

from sklearn.model_selection import StratifiedKFold

# Load the dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize the data
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# One-hot encode the target labels
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Define the model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)


outer_scores = []
for train_index, test_index in outer_cv.split(X_train, y_train.argmax(1)):
    X_train_outer, X_test_outer = X_train[train_index], X_train[test_index]
    y_train_outer, y_test_outer = y_train[train_index], y_train[test_index]

    inner_scores = []
    for inner_train_index, inner_test_index in inner_cv.split(X_train_outer, y_train_outer.argmax(1)):
        X_train_inner, X_test_inner = X_train_outer[inner_train_index], X_train_outer[inner_test_index]
        y_train_inner, y_test_inner = y_train_outer[inner_train_index], y_train_outer[inner_test_index]

        model = create_model()
        model.fit(X_train_inner.reshape(-1, 28, 28, 1), y_train_inner, epochs=5, batch_size=64, verbose=0)
        score = model.evaluate(X_test_inner.reshape(-1, 28, 28, 1), y_test_inner, verbose=0)
        inner_scores.append(score[1])

    outer_model = create_model()
    outer_model.fit(X_train_outer.reshape(-1, 28, 28, 1), y_train_outer, epochs=5, batch_size=64, verbose=0)
    score = outer_model.evaluate(X_test_outer.reshape(-1, 28, 28, 1), y_test_outer, verbose=0)
    outer_scores.append(score[1])

# Print the results
print('Inner CV scores:', inner_scores)
print('Outer CV scores:', outer_scores)
print('Mean outer CV score:', np.mean(outer_scores))

Inner CV scores: [0.8797810077667236, 0.8766219019889832, 0.8879472017288208]
Outer CV scores: [0.9006999731063843, 0.8948000073432922, 0.8912500143051147]
Mean outer CV score: 0.8955833315849304


In [2]:
acc = np.mean(outer_scores)
print(acc*100)

89.55833315849304
